<a href="https://colab.research.google.com/github/Peiprjs/voila/blob/main/notebooks/HIV_Deaths_VS_Total_expenditure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sexually transmitted diseases as indicators of sexual and reproductive healthcare in South Africa and the Netherlands
#### A project by Ellie Petrova (i6326413) and Mar Roca (i6351071)

---
> Has South Africa improved its treatment of pregnant and fertile people who are HIV-positive between 2000 and 2015, leading to less child deaths? Have Hepatitis-B vaccinations (Hep-B can be transmitted sexually) in 1-year-olds increased in that timeframe? Does increased healthcare spending correlate with lower HIV/AIDS deaths in newborns and children, as well as Hepatitis-B vaccinations in 1-year-olds?

> How does the death rate of 0-4 year-olds due to HIV/AIDS compare between the Netherlands and an ex-Dutch colony (South Africa)? Is it related to healthcare spending?

In [1]:
# REMOVE?? Only if stuff breaks
# !pip install altair==5.2.0 --quiet
# (we needed Altair 5.2.0 specifically because of some version-specific update, so we call for it using PIP, muting the output)

In [2]:
import numpy as np
import pandas as pd
import altair as alt
import ipywidgets as widgets
from ipywidgets import interact
# This imports the required dependencies

In [3]:
healthFactors = pd.read_csv('https://raw.githubusercontent.com/NHameleers/dtz2025-datasets/master/CountryHealthFactors.csv')
healthFactors = healthFactors.rename(columns=lambda x: x.strip().title())
healthFactors = healthFactors.rename(columns={"Hiv/Aids": "HIV/AIDS", "Gdp": "GDP", "Bmi": "BMI"})
# This imports the CSV with the dataset and strips leading and trailing whitespaces in the indexes. It also changes all indexes into the Title format (first letter of each word capitalised) for style and uniformity. HIV/AIDS, BMI and GDP are kept as exemptions due to being made up of initials.
hf_SouthAfrica = healthFactors.loc[healthFactors.Country == "South Africa", ['Year', 'Percentage Expenditure', 'HIV/AIDS', 'Hepatitis B']]
hf_Netherlands = healthFactors.loc[healthFactors.Country == "Netherlands", ['Year', 'Percentage Expenditure', 'HIV/AIDS', 'Hepatitis B']]
# This selects only the data that we're interested in: Years, Total expenditure and HIV/AIDS
# from rows which Country column is equal to South Africa and the Netherlands respectively

In [22]:
def make_scatter(dataframe, varX, varY, typeX):
  dataframe[varX] = dataframe[varX].astype(str)
  # We need to perform this type conversion because of Altair being weird with years. According to the documentation, we must define it as string and specify the variable type as temporal
  scatter = alt.Chart(dataframe).mark_circle(opacity=0.5).encode(
    alt.X(varX, type= typeX, scale=alt.Scale(zero=False)),
    alt.Y(varY, type= 'quantitative'),)
  # This first part draws the dots in the scatter plot
  scatter_w_loess = scatter + scatter.transform_loess(varX, varY).mark_line()
  # This second part draws a LOESS (LOcally Estimated Scatterplot Smoothing) line, which makes seeing the evolution easier.
  display(scatter_w_loess)
# This first class defines how to make a scatter plot with LOESS very easily, simply passing to the class a dataframe, the x-variable and the y-variable indexes.

def make_boxplot(dataframe, var):
  boxplot = alt.Chart(dataframe).mark_boxplot().encode(
    alt.Y(var, type = 'quantitative', scale=alt.Scale(zero=True))
  )
  display(boxplot)

We start off by importing all the necessary packages, and loading the CSV containing the health factors data. This file needs to be parsed using Pandas, and some indexes must be corrected. We strip leading and trailing spaces, as well as converting all indexes to a Title format (first letter of each word is capitalised) for stylistic and standardisation reasons, except for acronyms.
We also select the variables that we are interested in studying: Year, Percentage Expenditure and HIV/AIDS in the rows in which the country is South Africa or Netherlands. Finally, we set up some Classes that we will need to use frequently.

In [5]:
print(f"The datasets have both the same shape: {hf_SouthAfrica.shape[0]} rows and {hf_SouthAfrica.shape[1]} colums")
print(f"The data that we have was collected between {hf_SouthAfrica.Year.min()} and {hf_Netherlands.Year.max()}")

The datasets have both the same shape: 16 rows and 4 colums
The data that we have was collected between 2000 and 2015


By running `hf_SouthAfrica.shape` or `hf_Netherlands.shape` we get the shapes of the dataframes resulting from isolating the data that we are interested in. We can observe that both of the resulting frames have a shape of **16x3**: **16** rows and **3** columns. By running `hf_SouthAfrica.Year.min()` or `hf_SouthAfrica.Year.max()` we can find out between what years we have the data from: **2000** to **2015**

# In South Africa throughout the years (2000-2015) - Mar Roca


In this part of the report we will explore the following questions:
> Has South Africa improved its treatment of pregnant and fertile people who are HIV-positive between 2000 and 2015, leading to less child deaths? Does increased healthcare spending correlate with lower HIV/AIDS deaths in newborns and children?

We will try to answer these questions basing us on percentage of GDP/capita spent on healthcare, the per-mille deaths of children aged 0-4 attributed to HIV/AIDS, and the percentage of 1-year-olds vaccinated against Hepatitis-B. We believe these is the best variable we have available to determine the quality of sexual and reproductive healthcare, especially when it comes to sexually-transmitted diseases, as the parents can transfer HIV to their children, but only if they are HIV-Positive, therefore, not receiving antiretroviral treatment that would turn them HIV-Indetectable, and therefore HIV-Intransmittable (in the case of per-mille HIV/AIDS deaths of 0-4 year-olds); and a more robust healthcare system with good public conscientisation of the need for vaccinations (in the case of Hepatitis-B).

We hypothesize that we will see some sort of negative correlation between expenditure and HIV/AIDS deaths in newborns and children, because with more healthcare spending will come more testing availability (we may see a small increase of **REPORTED** HIV/AIDS deaths in newborns as healthcare spending increases, due to the increased availability of testing), as well as better access to antiretrovirals, which can turn a previously HIV-positive into an HIV-indetectable, meaning that HIV can no longer be transmitted, yet still lays dormant in the patient's body. We also hypothesize that HIV/AIDS deaths in children aged 0-4 will decrease as years pass, but that healthcare spending will increase. We also hope to see a similar trend with Hepatitis-B vaccinations for 1-year-olds: increasing vaccinations as expenditure and years increase.

In [ ]:
print(hf_SouthAfrica.loc[hf_SouthAfrica.isna().values])

The singular line of code above allows us to find in the entire dataframe of data that we selected the instances in which there's a NaN. We can see that in our selected portion of data, there is no missing values, as it returns an empty dataframe.

The first step in any statistical analysis is to graph the variables we're interested in studying, to see if there is and (in case there is) what kind of relationship the two variables follow.
So, we will start by generating graphs using Altair. The next step will be to calculate some descriptive statistics.

In [17]:
graph_HIV_Year = widgets.Output()
with graph_HIV_Year:
  make_scatter(hf_SouthAfrica, "Year", "HIV/AIDS", 'temporal')

graph_HEP_Year = widgets.Output()
with graph_HEP_Year:
  make_scatter(hf_SouthAfrica, "Year", "Hepatitis B", 'temporal')

graph_Expend_Year = widgets.Output()
with graph_Expend_Year:
  make_scatter(hf_SouthAfrica, "Year", "Percentage Expenditure", 'temporal')

graph_HIV_Expend = widgets.Output()
with graph_HIV_Expend:
  make_scatter(hf_SouthAfrica, "Percentage Expenditure", "HIV/AIDS", 'quantitative')

graph_HEP_Expend = widgets.Output()
with graph_HEP_Expend:
  make_scatter(hf_SouthAfrica, "Percentage Expenditure", "Hepatitis B", 'quantitative')
# This first section defines all the graphs we want to plot

widgets.HBox([graph_HIV_Year, graph_HEP_Year, graph_Expend_Year, graph_HIV_Expend, graph_HEP_Expend])
# And then we display all of them together in a row

The code above generates XY scatterplots for the variables that we are interested in, which show that:
- Except for an increase before 2004, HIV deaths in children aged 0 to 4 steadily decreased since 2004 and seem to have plateau'd at below 5 per 1000 live births.
- There seems to be two outliers in the percentage expenditure. We can see one in 2011, when South Africa *only* spent a bit over 100% of its GDP per capita, and in 2015, where it was reported at 0. We can assume that the data for 2015 is missing, and so the percentage calculation returned 0.
- HIV deaths seemed to increase with percentage expenditure, until the expenditure reached a certain limit, at which point the cases seem to drop significantly.
- Except for the case of two outliers, it would seem as though vaccinations of 1-year-olds have been kept uniformly high between 70% and 80%. We were unable to find information online as to why this sudden 2-year drop might have been caused, so we will assume it was due to some sort of informatic or bureaucratic mishap.

We will compute a new dataframe of percentage expenditure excluding the outlying values. They will appear as `NaN`, so they will be ignored when we perform descriptive statistics using `Pandas.describe()` or graphing using `Altair.Chart()`. We will then graph this new dataframe against the year.

In [20]:
hf_SouthAfrica["Percentage Expenditure Outlierless"] = hf_SouthAfrica.loc[(hf_SouthAfrica["Year"] != "2011") & (hf_SouthAfrica["Year"] != "2015"), "Percentage Expenditure"]
hf_SouthAfrica["Hepatitis B Outlierless"] = hf_SouthAfrica.loc[(hf_SouthAfrica["Year"] != "2013") & (hf_SouthAfrica["Year"] != "2014"), "Hepatitis B"]
# We use these line of code to remove the datapoints which we considered outliers.

graph_Expend_Year = widgets.Output()
with graph_Expend_Year:
  make_scatter(hf_SouthAfrica, "Year", "Percentage Expenditure Outlierless", 'temporal')

graph_HIV_Expend = widgets.Output()
with graph_HIV_Expend:
  make_scatter(hf_SouthAfrica, "Percentage Expenditure Outlierless", "HIV/AIDS", 'quantitative')

graph_HEP_Expend = widgets.Output()
with graph_HEP_Expend:
  make_scatter(hf_SouthAfrica, "Percentage Expenditure Outlierless", "Hepatitis B Outlierless", 'quantitative')

widgets.HBox([graph_Expend_Year, graph_HIV_Expend, graph_HEP_Expend])

In this new graph we can observe how expenditure on healthcare (as a percentage of GDP per capita) has been slowly increasing since 2000, with a small decline after 2012.
We can also observe in the Percentage Expenditure vs HIV/AIDS deaths graph where outliers have been removed, that there is actually a gradual increase as healthcare expenditure increases, and an immediate drop-off when it increases after a certain point.


We will now perform a descriptive statistics analysis on our data.

In [23]:
hf_SouthAfrica["Percentage Expenditure Outlierless"] = hf_SouthAfrica["Percentage Expenditure Outlierless"].astype('float64') # Had to fix Percentage Expenditure AND Percentage Expenditure Outlierless somehow being an object and not a float64

table = widgets.Output()
with table:
  display(hf_SouthAfrica.describe())

box_HIV = widgets.Output()
with box_HIV:
  make_boxplot(hf_SouthAfrica, 'HIV/AIDS')

box_PEO = widgets.Output()
with box_PEO:
  make_boxplot(hf_SouthAfrica, 'Percentage Expenditure Outlierless')

box_PE = widgets.Output()
with box_PE:
  make_boxplot(hf_SouthAfrica, 'Percentage Expenditure')

box_HEPBO = widgets.Output()
with box_HEPBO:
  make_boxplot(hf_SouthAfrica, 'Hepatitis B Outlierless')

box_HEPB = widgets.Output()
with box_HEPB:
  make_boxplot(hf_SouthAfrica, 'Hepatitis B')


widgets.HBox([table, box_PEO, box_PE, box_HIV, box_HEPBO, box_HEPB])

We can see from the descriptive statistics table above that Percentage Expenditure had two less valid values, as we removed two clear outliers. We can also see at a glance from looking at the boxplots that HIV/AIDS deaths in children aged 0-4 that in the time period we studied, there was a big range in numbers, though the median was quite high, indicating that any change happened late in the period, giving it a negative skew.

We can also see how removing the outliers affected the Percentage Expenditures, both Outliar-y and Outliar-less. We can also see that expenditure is quite consistent, and is skewed negatively as well.

In the case of Hepatitis B, we can see how we were able to remove the outliers from 2013 and 2014, as Altair marked them on the boxplot with two circles. Another circle appears on the outlierless computed variable, but that is not unexpected, and it's not as extreme an outlier as the two values we removed.

# Comparing South Africa and The Netherlands in [year] - Ellie Petrova

